## Base de données JSON : MongoDB

### Installation et configuration

https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-windows/

Téléchargement du package d'installation de mongoDB 6.0.2 community server for Windows
https://www.mongodb.com/try/download/community

Eventuellement télécharger le shell (ou pas)
https://www.mongodb.com/try/download/shell


Exemple de commande à exécuter pour le lancement de mongod.exe s'il n'a pas été configuré comme un service

In [1]:
from pymongo import MongoClient

<div id="client"></div>

### Instantiation d'un client

In [2]:
client = MongoClient('localhost', 27017)
print(client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [3]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'config', 'sizeOnDisk': 73728, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728, 'empty': False}
{'name': 'test', 'sizeOnDisk': 196608, 'empty': False}


<div id="list_collections"></div>

### Base de données

In [4]:
db = client['test']
db.list_collection_names()

['transiliens', 'transiliens:mesures', 'people']

### Collection

In [5]:
# Collection
people = db['people']

### Document

<div id="insert_one" style="font-size:120%; border-bottom:1px solid #444; background-color:#eee">Create</div>

In [6]:
person = people.find_one({"first_name":"Raymond"})
if person:
    person_id = person['_id']
else:
    person = {
        "first_name": "Raymond",
        "last_name": "Deubaze",
        "email": "raymond.deubaze@ec-lyon.fr"
    }
    person_id = people.insert_one(person).inserted_id

In [7]:
import pprint

pprint.pprint(person)

assert person_id == person['_id']

{'_id': ObjectId('637785c399a65486d33db0e7'),
 'email': 'raymond.deubaze@ec-lyon.fr',
 'first_name': 'Raymond',
 'last_name': 'Deubaze'}


<div id="insert_many">Insertion multiple</div>

In [8]:
list_of_people = [
    {
        "first_name": "Jeanne",
        "last_name": "Aulapin",
        "email": "jan@provider.net"
    },
    {
        "first_name": "Anna",
        "last_name": "Conda",
        "email": "aca@provider.net"
    },
    {
        "first_name": "Jean",
        "last_name": "Bonnot",
        "email": "jbt@gmail.com"
    },
    {
        "first_name": "Alex",
        "last_name": "Terrieur",
        "email": "axtr@gmail.com"
    },
    {
        "first_name": "Alain",
        "last_name": "Terrieur",
        "email": "antr@gmail.com"
    },
    {
        "first_name": "Ginette",
        "last_name": "Ringard",
        "email": "gigi@wanadoo.fr"
    }
]

In [9]:
jeanne = people.find_one({"first_name":"Jeanne"})
if not jeanne:
    result = people.insert_many(list_of_people)
    pprint.pprint(result.inserted_ids)

In [10]:
pprint.pprint(list_of_people)

[{'email': 'jan@provider.net', 'first_name': 'Jeanne', 'last_name': 'Aulapin'},
 {'email': 'aca@provider.net', 'first_name': 'Anna', 'last_name': 'Conda'},
 {'email': 'jbt@gmail.com', 'first_name': 'Jean', 'last_name': 'Bonnot'},
 {'email': 'axtr@gmail.com', 'first_name': 'Alex', 'last_name': 'Terrieur'},
 {'email': 'antr@gmail.com', 'first_name': 'Alain', 'last_name': 'Terrieur'},
 {'email': 'gigi@wanadoo.fr', 'first_name': 'Ginette', 'last_name': 'Ringard'}]


<div id="read" style="font-size:120%; border-bottom:1px solid #444; background-color:#eee">Read</div>

In [11]:
raymond = people.find_one()
pprint.pprint(raymond)

{'_id': ObjectId('637785c399a65486d33db0e7'),
 'email': 'raymond.deubaze@ec-lyon.fr',
 'first_name': 'Raymond',
 'last_name': 'Deubaze'}


In [12]:
all = people.find()
for p in all:
    print(p['first_name'] + ' ' + p['last_name'])

Raymond Deubaze
Jeanne Aulapin
Anna Conda
Jean Bonnot
Alex Terrieur
Alain Terrieur
Ginette Ringard


<div id="filter" style="font-size:120%; border-bottom:1px solid #444; background-color:#eee">Read and Filter</div>

In [13]:
deubaze = people.find_one({'last_name': 'Deubaze'})
pprint.pprint(deubaze)

{'_id': ObjectId('637785c399a65486d33db0e7'),
 'email': 'raymond.deubaze@ec-lyon.fr',
 'first_name': 'Raymond',
 'last_name': 'Deubaze'}


<div id="filter_by_strid"></div>

In [14]:
p = people.find_one({'_id': deubaze['_id']})
pprint.pprint(p)

{'_id': ObjectId('637785c399a65486d33db0e7'),
 'email': 'raymond.deubaze@ec-lyon.fr',
 'first_name': 'Raymond',
 'last_name': 'Deubaze'}


In [15]:
id = str(deubaze['_id'])
print(id)

p = people.find_one({'_id': id })
print(p)

637785c399a65486d33db0e7
None


<div id="filter_by_oid"></div>

In [16]:
from bson.objectid import ObjectId

p = people.find_one({'_id': ObjectId(id)})
pprint.pprint(p)

{'_id': ObjectId('637785c399a65486d33db0e7'),
 'email': 'raymond.deubaze@ec-lyon.fr',
 'first_name': 'Raymond',
 'last_name': 'Deubaze'}


In [17]:
family = people.find({"last_name": 'Terrieur'})
for member in family:
    print(member['first_name'] + ' ' + member['last_name'])

Alex Terrieur
Alain Terrieur


<div style="font-size:120%; border-bottom:1px solid #444; background-color:#eee">Count</div>

In [18]:
print(people.count_documents({}))
print(people.count_documents({'last_name': 'Terrieur'}))

7
2


In [19]:
print(people.count_documents({'first_name': 'Raymond'}))

1


<div id="update" style="font-size:120%; border-bottom:1px solid #444; background-color:#eee">Update</div>

In [20]:
retval = people.update_one(
    {'first_name':'Ginette'},
    { '$set': {'first_name':'Gigi'},
      '$unset': {'aka':''}
    }
)
print(retval.raw_result)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


In [21]:
ginette = people.find_one({'first_name': 'Gigi'})
print(ginette['first_name'])

Gigi


In [22]:
ginette['first_name'] = 'Ginette'
retval = people.update_one({'_id': ginette['_id']},{'$set': ginette})
print(retval.raw_result)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


In [23]:
retval = people.update_one(
    {'_id': ginette['_id']},
    {'$set': {'aka': 'Gigi'}}
)
print(retval.raw_result)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


In [24]:
gigi = people.find_one({'first_name': 'Ginette'})
pprint.pprint(gigi)

{'_id': ObjectId('63778677be21fce9c982c586'),
 'aka': 'Gigi',
 'email': 'gigi@wanadoo.fr',
 'first_name': 'Ginette',
 'last_name': 'Ringard'}


In [25]:
retval = people.update_one({'_id': ginette['_id']}, {'$set': ginette})
print(retval.raw_result)

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}


<p id="replace">
La commande précédente met uniquement à jour les attributs communs avec ginette, donc ne fait rien dans le cas présent (nModified = 0).
</p>
<p>
Contrairement à la commande suivante, qui remplace complètement le document existant avec le document fourni, ce qui revient dans le cas présent à supprimer l'attribut 'aka'.
</p>

In [26]:
retval = people.replace_one({'_id': ginette['_id']}, ginette)
print(retval.raw_result)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


<div id="update_many"></div>

In [27]:
retval = people.update_many(
    {'last_name': 'Terrieur'},
    {'$set': {'address': 'Ecully'}}
)
print(retval.raw_result)

{'n': 2, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}


<div id="delete" style="font-size:120%; border-bottom:1px solid #444; background-color:#eee">Delete</div>

In [28]:
does = [
    { 'first_name': 'John', 'last_name': 'Doe'},
    { 'first_name': 'Jane', 'last_name': 'Doe'},
    { 'first_name': 'Baby', 'last_name': 'Doe'}
]
people.insert_many(does)

In [29]:
retval = people.delete_one({'first_name': 'Baby'})
print(retval.raw_result)

{'n': 1, 'ok': 1.0}


In [30]:
retval = people.delete_many({'last_name': 'Doe'})
print(retval.raw_result)

{'n': 2, 'ok': 1.0}


In [31]:
deubazes = people.find({'first_name': 'Raymond'})
for p in deubazes[1:]:
    people.delete_one(p)
print(people.count_documents({'first_name': 'Raymond'}))

1


<div id="query_operators"></div>

### Opérateurs pour les requêtes

In [32]:
friends = people.find({ 'first_name': {'$in': ['Alain','Alex']}})
for p in friends:
    print(p['first_name'] + ' ' + p['last_name'])

Alex Terrieur
Alain Terrieur


In [33]:
girls = people.find({ '$or': [
    {'first_name': 'Anna'},
    {'last_name': 'Aulapin'},
    {'email': 'gigi@wanadoo.fr'}
]})
for p in girls:
    print(p['first_name'] + ' ' + p['last_name'])                     

Jeanne Aulapin
Anna Conda
Ginette Ringard


In [75]:
raymond = people.find_one({ '$and': [
    {'first_name': 'Raymond'},
    {'last_name': 'Deubaze'},
]})
print(raymond['email'])

raymond.deubaze@ec-lyon.fr


In [76]:
raymond = people.find_one({'first_name': 'Raymond', 'last_name': 'Deubaze'})
print(raymond['email'])

raymond.deubaze@ec-lyon.fr


<div id="regexp"></div>

In [34]:
import re

boys = people.find({
    '$or' : [
        {'first_name': 'Raymond'},
        {'email': { '$regex': re.compile('gmail.com')}}
    ]
})
for p in boys:
    print(p['first_name'] + ' ' + p['last_name'])                     

Raymond Deubaze
Jean Bonnot
Alex Terrieur
Alain Terrieur


In [35]:
all = people.find()
for p in all:
    print(p)

{'_id': ObjectId('637785c399a65486d33db0e7'), 'first_name': 'Raymond', 'last_name': 'Deubaze', 'email': 'raymond.deubaze@ec-lyon.fr'}
{'_id': ObjectId('63778677be21fce9c982c581'), 'first_name': 'Jeanne', 'last_name': 'Aulapin', 'email': 'jan@provider.net'}
{'_id': ObjectId('63778677be21fce9c982c582'), 'first_name': 'Anna', 'last_name': 'Conda', 'email': 'aca@provider.net'}
{'_id': ObjectId('63778677be21fce9c982c583'), 'first_name': 'Jean', 'last_name': 'Bonnot', 'email': 'jbt@gmail.com'}
{'_id': ObjectId('63778677be21fce9c982c584'), 'first_name': 'Alex', 'last_name': 'Terrieur', 'email': 'axtr@gmail.com', 'address': 'Ecully'}
{'_id': ObjectId('63778677be21fce9c982c585'), 'first_name': 'Alain', 'last_name': 'Terrieur', 'email': 'antr@gmail.com', 'address': 'Ecully'}
{'_id': ObjectId('63778677be21fce9c982c586'), 'first_name': 'Ginette', 'last_name': 'Ringard', 'email': 'gigi@wanadoo.fr'}


<div id="datetime"></div>
### Date, Timestamp et types spécialisés
#### Date

Avec le driver PyMongo, les dates et les timestamp s'utilisent via datetime, le module standard de python.

In [36]:
import datetime

# Date
date = datetime.datetime(2002,11,24,17,53,0)
retval = people.update_one({'first_name': 'Raymond'}, {'$set': {'birthdate': date}})
print(retval.raw_result)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


In [38]:
raymond = people.find_one({'first_name': 'Raymond'})
print(raymond)

{'_id': ObjectId('637785c399a65486d33db0e7'), 'first_name': 'Raymond', 'last_name': 'Deubaze', 'email': 'raymond.deubaze@ec-lyon.fr', 'birthdate': datetime.datetime(2002, 11, 24, 17, 53)}


In [63]:
print(people.find_one({'_id':raymond['_id']},{'birthdate':1,'_id':0}))

{'birthdate': datetime.datetime(2002, 11, 24, 17, 53)}


In [40]:
print(type(raymond['birthdate']), raymond['birthdate'])

<class 'datetime.datetime'> 2002-11-24 17:53:00


#### Timestamp

now = datetime.datetime.utcnow()
retval = people.update_one({'first_name': 'Raymond'}, {'$set': {'last_updated': now}})
print(retval.raw_result)

In [49]:
raymond = people.find_one({'first_name': 'Raymond'})
print(type(raymond['last_updated']), raymond['last_updated'])

<class 'datetime.datetime'> 2022-11-24 17:22:47.717000


<div id="objectid"></div>
#### Object ID

In [41]:
from bson.objectid import ObjectId

print(type(raymond['_id']), raymond['_id'])

<class 'bson.objectid.ObjectId'> 637785c399a65486d33db0e7


<div id="decimal128"></div>
#### Decimal 128

In [57]:
from bson.decimal128 import Decimal128

retval = people.update_one({'first_name': 'Raymond'}, {'$set': {'credit': Decimal128('123456789.12')}})
print(retval.raw_result)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


In [58]:
raymond = people.find_one({'first_name': 'Raymond'})
print(type(raymond['credit']), raymond['credit'])

<class 'bson.decimal128.Decimal128'> 123456789.12


<div id="projection"></div>

#### Requêtes avec projection

In [72]:
all = people.find({},{"first_name": 1, "last_name":1, "_id":0})
for p in all: pprint.pprint(p)

{'first_name': 'Raymond', 'last_name': 'Deubaze'}
{'first_name': 'Jeanne', 'last_name': 'Aulapin'}
{'first_name': 'Anna', 'last_name': 'Conda'}
{'first_name': 'Jean', 'last_name': 'Bonnot'}
{'first_name': 'Alex', 'last_name': 'Terrieur'}
{'first_name': 'Alain', 'last_name': 'Terrieur'}
{'first_name': 'Ginette', 'last_name': 'Ringard'}


In [37]:
#people.delete_many({})